# Assignment 1: ML Engineering 1

In [ ]:
# enter your name and UFL email address
name = 'enter your name'
email = 'enter your email'

In [ ]:
if name == 'enter your name' or email == 'enter your email':
    assert False, 'Enter your name & email first!'
else:
    print('Assignment 1 -- name: {}, email: {}\n'.format(name, email))
    
    # Load packages we need
    import sys
    import os
    import time
    import tempfile 

    import numpy as np
    import pandas as pd

    from matplotlib import pyplot as plt
    plt.rcParams.update({'font.size': 16})

    import tensorflow as tf


    # Let's check our software versions
    print('### Python version: ' + __import__('sys').version)
    print('### TF version: ' + tf.__version__)
    print('### Keras version: ' + tf.keras.__version__)
    print('------------')


    # load our packages / code
    sys.path.insert(1, '../common/')
    import utils
    import plots

In [ ]:
# global parameters to control behavior of the pre-processing, ML, analysis, etc.
seed = 17

# deterministic seed for reproducibility
np.random.seed(seed)

## Part 1: Loading and Pre-processing Data

### Loading data

In [ ]:
# Use pandas to load the data from compressed CSV
df1 = pd.read_csv('../data/adult.data.gz', compression='gzip', header=0, na_values='?', sep=' *, *', skipinitialspace=True, engine='python')
df2 = pd.read_csv('../data/adult.test.gz', compression='gzip', header=0, na_values='?',sep=' *, *', skipinitialspace=True, engine='python')

In [ ]:
# Check that we loaded the data as expected
df1_expected_shape = (32561,15)
df2_expected_shape = (16281,15)

assert df1.shape == df1_expected_shape, 'Unexpected shape of df1!'
assert df2.shape == df2_expected_shape, 'Unexpected shape of df2!'

In [ ]:
# Merge df1 and df2 for pre-processing and cleaning
dfraw = df1.append(df2, ignore_index=True, sort=False)
assert dfraw.shape == (df1_expected_shape[0]+df2_expected_shape[0],15)

### Cleaning & pre-processing data

In [ ]:
## header right now: age (num), workclass (cat), fnlwgt (num), education (cat) education-num (num), marital-status (cat), occupation (cat), relationship (cat), race (cat), sex (cat), capital-gain (num), capital-loss (num), hours-per-week (num), native-country (cat), income (cat)
dfraw.columns

In [ ]:
# Let's start cleaning up the data
df = dfraw.copy()

# Let's get rid of the record weights 'fnlwgt' and 'education' (which contains same information as 'education-num')
if 'fnlwgt' in df.columns:
    df = df.drop('fnlwgt', axis=1)
if 'education' in df.columns:
    df = df.drop('education', axis=1)

# remove rows with NaN
df = df.dropna(axis=0)

In [ ]:
# Now we need to recode the categorical attributes
workclass_list = ['Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay']

marital_status_list = ['Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed']

occupation_list = ['Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving' ]
    
relationship_list = ['Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife']

race_list =['Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White' ]

sex_list = ['Male', 'Female']

native_country_list = ['Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic',
        'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece',
        'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary',
        'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico',
        'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland',
        'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand',
        'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia']

income_list = ['<=50K', '>50K']

cat_cols = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
cat_lists = [workclass_list, marital_status_list, occupation_list, relationship_list, race_list, sex_list, native_country_list, income_list]

# first make all the numerical cols integers
is_cat = []
for col_name in df.columns:
    cat = True
    if not col_name in cat_cols:
        df[col_name] = df[col_name].astype(float)
        cat = False
    is_cat.append(cat)

# encode categorical
for i, col_name in enumerate(cat_cols):
    df[col_name] = df[col_name].apply(lambda v: cat_lists[i].index(v))
    df[col_name] = df[col_name].astype(int)
    
# finally, make education-num 0 based and integer
df['education-num'] = (df['education-num']-1).astype(int)
    
    
is_cat = [True if col_name in cat_cols else False for col_name in df.columns]
is_cat[2] = True # education num
col_names = df.columns
col_names = [c for c in df.columns]

## [Task 1] (15 points) Let's look at our data!

### [Task 1a] (5 points) Display some info about our DataFrame (df).

In [ ]:
###* Put your code below (hint: 1 line) *###


### [Task 1b] (5 points) Show the first 8 rows of our DataFrame (df).

In [ ]:
###* Put your code below (hint: 1 line, be sure that you display exactly the first 8 rows) *###


### [Task 1c] (5 points) Display the shape of our DataFrame (df). 

In [ ]:
# how many records do we have?
###* Put your code below (hint: 1 line) *###


### Feature engineering

In [ ]:
# grab all the data as a numpy array
all_xy = np.asarray(df, dtype='float64')
assert all_xy.shape[1] == 13

label_col_idx = all_xy.shape[1]-1
features_col_idx = range(0, label_col_idx)

feature_names = col_names[0:label_col_idx]

In [ ]:
# separate features from the label
all_x = all_xy[:,features_col_idx]
all_y = all_xy[:,label_col_idx]

is_cat_features = is_cat[0:len(is_cat)-1]

## [Task 2] (35 points) Feature Engineering

### [Task 2a] (10 points) Standardize (i.e., z-score normalize) the numerical features of 'all_x', store the results (all features, all instances) in 'all_x_zscore'.

In [ ]:
###* Put your code below. Be sure to store the result back into 'all_x_zscore' (make a copy, do not modify 'all_x')! *###
###* Hint: use is_cat_features to determine the features to standardize *###
###* Note: you can use scikit-learn or you can do it manually with numpy. *###
###* If you do it manually, it is recommended to put your code in the utils.py file (name your function 'zscore_normalize') *###




### [Task 2b] (10 points) Normalize (i.e., min-max normalize) the numerical features of 'all_x', store the results (all features, all instances) in 'all_x_normalized'.

In [ ]:
###* Put your code below. Be sure to store the result back into 'all_x_normalized' (make a copy, do not modify 'all_x')! *###
###* Hint: use is_cat_features to determine the features to normalize *###
###* Note: you can use scikit-learn or you can do it manually with numpy. *###
###* If you do it manually, it is recommended to put your code in the utils.py file (name your function 'minmax_normalize') *###




### [Task 2c] (10 points) Convert the categorical features to binary features (one-hot encoding) of 'all_x_zscore' and 'all_x_normalized'. Store the results (all features, all instances) in 'all_x_zscore_onehot' and 'all_x_normalized_onehot'.

In [ ]:
###* Put your code below. Be sure to store the result back into 'all_x_zscore_onehot' and 'all_x_normalized' (make a copy, do not modify 'all_x')! *###
###* Hint: use is_cat_features to determine the features to one-hot encode *###
###* Note: you can use scikit-learn or you can do it manually with numpy. *###
###* If you do it manually, it is recommended to put your code in the utils.py file (name your function 'one_hot_encode') *###



### [Task 2d] (5 points) Sanity check of the mean and standard deviation of 'all_x_zscore', the min/max of 'all_x_normalized'

In [ ]:
# sanity check the means and std, min/max
###* Put your code below. Display the mean, standard deviation, min, max of a few (e.g., 3 or 4) 
### of features (e.g., those normalized, standardized, one-hot encoded) *###



### [Task 2d] What do you notice? Are the mean and standard deviation as you expected? What about the min/max? Why or why not?

In [ ]:
###* Put your answer as a comment *###
#
#
#

### Train, Test, Validation Split

In [ ]:
# now split between train, test, and validation
prop_vec = [14, 3, 3]

# what preprocessed features are we using?
t = 'normalized'  # do not editing this line when answering task 3

def grab_data(t):
    if t == 'zscore':
        features_x = all_x_zscore
    elif t == 'normalized':
        features_x = all_x_normalized
    elif t == 'onehot_z':
        features_x = all_x_zscore_onehot
    elif t == 'onehot_n':
        features_x = all_x_normalized_onehot
    else:
        features_x = all_x

    # note: because of the deterministic split, we'll always be looking at the same records for train/val/test, 
    # no matter which preprocessed features set we look at
    train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(features_x, all_y, prop_vec, 
                                                                                shuffle=True, seed=seed-1)
    return train_x, train_y, test_x, test_y, val_x, val_y


##-- Note: for task 3, we use the 'normalized' version of the data!
train_x, train_y, test_x, test_y, val_x, val_y = grab_data(t)

In [ ]:
# sanity check shapes
train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape

### Stats & Looking at the data

In [ ]:
# what does the distribution of labels look like?
utils.print_array_hist(train_y, label=col_names[-1])

In [ ]:
# what does the distribution of features look like?
for i in range(train_x.shape[1]):
    if is_cat_features[i]:
        utils.print_array_hist(train_x[:, i], label=col_names[i])
    else:
        utils.print_array_basic_stats(train_x[:, i], label=col_names[i])
    print()

In [ ]:
# plot feature distribution based on income
feature_idx = 0 # age

if t == 'zscore':
    bins = np.linspace(-2, 4, 40)
elif t == 'normalized':
    bins = np.linspace(0, 1, 20)
else:
    bins = 20

inc0_idx = train_y == 0
inc1_idx = train_y == 1

plt.hist(train_x[inc0_idx,feature_idx], bins, density=True, alpha=0.5, label=income_list[0])
plt.hist(train_x[inc1_idx,feature_idx], bins, density=True, alpha=0.5, label=income_list[1])

plt.xlabel('{} {}'.format(col_names[feature_idx], t))
plt.ylabel('Density')

plt.legend(loc='upper right')
plt.show()

## [Task 3] (20 points): Correlations

### [Task 3a] (5 points) Plot a heatmap of (pairwise) correlation between features and labels (see exercise 1)

In [ ]:
### Hint: make sure you compute all pairwise correlation and use plots.heatmap()! ###
###* Put your code here *###



### [Task 3b] (10 points) What do you observe? Which features are correlated with the label?

In [ ]:
###* Put your answer as a comment here ###
#
#
#
#

### [Task 3c] (5 points) Given that some features like workclass, marital-status, race, sex are categorical, is the Pearson correlation coefficient a good measure of statistical relationship? If yes, explain why. If not, explain why not *and* propose (in brief) an alternative.

In [ ]:
###* Put your answer as a comment here ###
### If you need to refer to some external resource/link, please cite it properly ###
#
#
#
#

## Part 2: Let's train some models!

## [Task 4] (30 points): Learning Curves

### [Task 4a] (10 points): Complete the definition of the subsample_train_eval function which repreatedly trains a model using a subset of the train data and calculates training & validation accuracy

In [ ]:
from sklearn.base import clone

def model_accuracy(model, x, true_y):
    pred = model.predict(x)
    return np.sum(pred == true_y) / true_y.shape[0]

###* Put your code in the function below *###
def subsample_train_eval(model, train_x, train_y, val_x, val_y, sample_size, num_repeats=10):
    
    train_accs = np.zeros((num_repeats,1))
    val_accs = np.zeros_like(train_accs)
    
    rng = np.random.default_rng()
    for i in range(0, num_repeats):
        m = clone(model) # clone the model
        
        # take 'sample_size' random records from train_x (and train_y) and fit model m on it
        # make sure your code gives you a different random subset each time (hint: use 'rng')!
        ###* Put your code here *###
       
        
        
    
        ### store the sample dataset in 'tr_x' and 'tr_y'
        
        # get the accuracies
        train_accs[i] = model_accuracy(m, tr_x, tr_y)
        val_accs[i] = model_accuracy(m, val_x, val_y)
        
    return train_accs, val_accs

In [ ]:
def plot_learning_curve(model_instance, model_name, train_x, train_y, val_x, val_y, train_szs=None):
    
    if train_szs is None:
        train_szs = np.array([25, 50, 100, 200, 300, 500, 750, 1000, 1500, 2000])
        
    num_datapoints = train_szs.shape[0]
    accs = np.zeros((num_datapoints,2))
    laccs = np.zeros((num_datapoints,2))
    uaccs = np.zeros((num_datapoints,2))
    
    sys.stdout.write('Plotting')

    for i, tsz in enumerate(train_szs):
        
        nr = 20 if tsz <= 1000 else 10
        train_accs, val_accs = subsample_train_eval(model_instance, train_x, train_y, val_x, val_y, tsz, num_repeats=nr)
        
        train_acc = np.mean(train_accs)
        train_acc_std = np.std(train_accs)
        
        accs[i,0] = train_acc
        laccs[i,0] = train_acc - train_acc_std
        uaccs[i,0] = train_acc + train_acc_std
        
        val_acc = np.mean(val_accs)
        val_acc_std = np.std(val_accs)
        
        accs[i,1] = val_acc
        laccs[i,1] = val_acc - val_acc_std
        uaccs[i,1] = val_acc + val_acc_std
        
        sys.stdout.write('.')
        sys.stdout.flush()
    print()
    
    fig, ax = plt.subplots(figsize=(15,8))

    plt.fill_between(train_szs, laccs[:,0]*100.0, uaccs[:,0]*100.0, color='r', alpha=0.3)
    plt.fill_between(train_szs, laccs[:,1]*100.0, uaccs[:,1]*100.0, color='b', alpha=0.3)
    
    plt.plot(train_szs, accs[:,0]*100.0, 'ro-', linewidth=3, label='Training')
    plt.plot(train_szs, accs[:,1]*100.0, 'bs--', linewidth=3, label='Validation')

    ax.set_xlim(0, np.max(train_szs))
    ax.set_ylim(65, 104)
    ax.set_xlabel('Number of training examples')
    ax.set_ylabel('Accuracy')
    ax.set_title('Learning Curves for {}'.format(model_name))

    plt.legend()

    plt.show()

### [Task 4b] (10 points): Complete the code of the cell below to plot the learning curves

In [ ]:
# Let's import what we need from scikit-learn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC

# grab the zscore normalized version of the data
train_x, train_y, test_x, test_y, val_x, val_y = grab_data('zscore')

###* put your code here: instantiate a linear SVM model (SVC with a linear kernel) with default hyperparameters and plot its learning curve *###
###* note: you just need to call 'plot_learning_curve' with the correct arguments

    

###* put your code here: instantiate a KNN (k=1) model with default hyperparameters and plot its learning curve *###
###* note: again, you just need to call 'plot_learning_curve' with the correct arguments



### [Task 4c] (10 points): What can you conclude about the size of the training data necessary for both the SVM model and the KNN model? If you had to choose between the SVM and the KNN, which would you pick and why?

In [ ]:
###* Put your answer as a comment ###
#
#
#
#

## [CIS6930 Additional Task -- Task 5] (25 points): Analysis of feature engineering

### [Task 5a] (10 points): Train SVM and kNN models as in Task 4 for min-max normalized and zscore normalized features (not one-hot encoded) and compare them (print accuracies, bar plot, whatever you think is appropriate). Use the the training set (no learning curves). What do you conclude and why?

In [ ]:
# Note: you can use evaluate_model() provided above (or you can do something else) for Tasks 5a and/or 5b #
def evaluate_model(name, model, train_x, train_y, val_x, val_y):
    train_acc = model_accuracy(model, train_x, train_y)
    val_acc = model_accuracy(model, val_x, val_y)
    print('[{}] Training accuracy: {:.2f}%, Validation accuracy: {:.2f}%'.format(name, train_acc*100, val_acc*100))
    

###* Put your code here *###
###* The SVM model: SVC(kernel='linear')
###* The KNN model: KNeighborsClassifier(n_neighbors=1)
###* Use 'grab_data' to get the feature engineered data you need  *###
#





In [ ]:
###* Put your answer (what do you conclude and why?) as a comment here ###
#
#
#
#

### [Task 5b] (15 points): Now repeat the analysis but this time compare the one-hot encoded features to non-one-hot encoded features. Use the full training set (no learning curves). What is the impact on the models (for both SVM & kNN) of one-hot encoding? Briefly comment on your findings.

In [ ]:
###* Put your code here *###
#
#






In [ ]:
###* Put your answer (i.e., comment on your findings) as a comment ###
#
#
#